# MIMIC-III Introduction Workshop

### Workshop coordinators: Dr Matthieu Komorowski, Dr Shabbir Syed-Abdul

In this section, we'll explore the data of a single patient, before building a simple predictive model.

# PART 1: Exploring the trajectory of a single patient

### Import Python libraries

We first need to import some tools for working with data in Python. 
- NumPy is for working with numbers
- Pandas is for analysing data
- MatPlotLib is for making plots
- Sqlite3 to connect to the database

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

### Connect to the database

- We can use the sqlite3 library to connect to the MIMIC database
- **Update the file path to your local folder!!**
- Once the connection is established, we'll run a simple SQL query.

In [ ]:
# Connect to the MIMIC database
conn = sqlite3.connect('C:\Users\Matthieu\mimic_workshop-master\data\mimicdata.sqlite')

In [ ]:
# Create our test query
test_query = """
SELECT subject_id, hadm_id, admittime, dischtime, admission_type, diagnosis
FROM admissions
"""

In [ ]:
# Run the query and assign the results to a variable
test = pd.read_sql_query(test_query,conn)

In [ ]:
# Display the first few rows
test.head()

### Load the chartevents data

- The chartevents table contains data charted at the patient bedside. It includes variables such as heart rate, respiratory rate, temperature, and so on.
- We'll begin by loading the chartevents data for a single patient.

In [ ]:
query = """
SELECT de.icustay_id
  , round((strftime('%s',de.charttime)-strftime('%s',ie.intime))/60.0/60.0+0.5) as HOURS
  , di.label
  , de.value
  , de.valuenum
  , de.uom
FROM chartevents de
INNER join d_items di
ON de.itemid = di.itemid
INNER join icustays ie
ON de.icustay_id = ie.icustay_id
WHERE de.icustay_id = 252522
ORDER BY charttime;
"""

ce = pd.read_sql_query(query,conn)


# OPTION 2: load chartevents from a CSV file
# ce = pd.read_csv('data/example_chartevents.csv', index_col='HOURSSINCEADMISSION')

In [ ]:
# Preview the data
# Use 'head' to limit the number of rows returned
ce.head()

### Review the patient's heart rate

- We can select individual columns using the column name. 
- For example, if we want to select just the label column, we write **```ce.LABEL```** or alternatively **```ce['LABEL']```**

In [ ]:
# Select a single column from the chartevents data we extracted 
ce['LABEL']

- In a similar way, we can select rows from data using indexes. 
- For example, to select rows where the label is equal to 'Heart Rate', we would create an index using **```[ce.LABEL=='Heart Rate']```** 

In [ ]:
# Select just the heart rate rows using an index
ce[ce.LABEL=='Heart Rate']

#### Plot 1: How did the patients heart rate change over time?

- Using the methods described above to select our data of interest, we can create our x and y axis values to create a time series plot of heart rate.

In [ ]:
# Which time stamps have a corresponding heart rate measurement?
print ce.index[ce.LABEL=='Heart Rate']

In [ ]:
# Set x equal to the times
x_hr = ce.HOURS[ce.LABEL=='Heart Rate']

# Set y equal to the heart rates
y_hr = ce.VALUENUM[ce.LABEL=='Heart Rate']

# Plot time against heart rate
plt.figure(figsize=(14, 6))
plt.plot(x_hr,y_hr)


plt.xlabel('Time',fontsize=16)
plt.ylabel('Heart rate',fontsize=16)
plt.title('Heart rate over time from admission to the intensive care unit')

#### Task 1

* What is happening to this patient's heart rate?
* Plot respiratory rate over time for the patient.
* Is there anything unusual about the patient's respiratory rate?


In [ ]:
# Exercise 1 here



### Did the patient's vital signs breach any alarm thresholds?

- Alarm systems in the intensive care unit are commonly based on high and low thresholds defined by the carer.
- False alarms are often a problem and so thresholds may be set arbitrarily to reduce alarms.
- As a result, alarm settings carry limited information.

#### Plot 2: Respiratory rate and alarms

In [ ]:
plt.figure(figsize=(14, 6))

plt.plot(ce.HOURS[ce.LABEL=='Respiratory Rate'], 
         ce.VALUENUM[ce.LABEL=='Respiratory Rate'],
         'k+', markersize=10, linewidth=4)

plt.plot(ce.HOURS[ce.LABEL=='Resp Alarm - High'], 
         ce.VALUENUM[ce.LABEL=='Resp Alarm - High'],
         'm--')

plt.plot(ce.HOURS[ce.LABEL=='Resp Alarm - Low'], 
         ce.VALUENUM[ce.LABEL=='Resp Alarm - Low'],
         'm--')

plt.xlabel('Time',fontsize=16)
plt.ylabel('Respiratory rate',fontsize=16)
plt.title('Respiratory rate over time from admission, with upper and lower alarm thresholds')
plt.ylim(0,55)


#### Task 2

- Based on the data, does it look like the alarms would have triggered for this patient?


### What was the patient's level of consciousness?

- Glasgow Coma Scale (GCS) is a measure of consciousness.
- It is commonly used for monitoring patients in the intensive care unit. 
- It consists of three components: eye response; verbal response; motor response.

In [ ]:
# Display the first few rows of the GCS eye response data
ce[ce.LABEL=='GCS - Eye Opening'].head()

#### Plot 3: GCS over time

In [ ]:
# Prepare the size of the figure
plt.figure(figsize=(18, 10))

# Set x equal to the times
x_hr = ce.HOURS[ce.LABEL=='Heart Rate']

# Set y equal to the heart rates
y_hr = ce.VALUENUM[ce.LABEL=='Heart Rate']


plt.plot(x_hr,y_hr)

plt.plot(ce.HOURS[ce.LABEL=='Respiratory Rate'], 
         ce.VALUENUM[ce.LABEL=='Respiratory Rate'],
         'k', markersize=6)

# Add a text label to the y-axis
plt.text(-20,155,'GCS - Eye Opening',fontsize=14)
plt.text(-20,150,'GCS - Motor Response',fontsize=14)
plt.text(-20,145,'GCS - Verbal Response',fontsize=14)   

# Iterate over list of GCS labels, plotting around 1 in 10 to avoid overlap
for i, txt in enumerate(ce.VALUE[ce.LABEL=='GCS - Eye Opening'].values):
    if np.mod(i,6)==0 and i < 65:
        plt.annotate(txt, (ce.HOURS[ce.LABEL=='GCS - Eye Opening'].values[i],155),fontsize=14)
        
for i, txt in enumerate(ce.VALUE[ce.LABEL=='GCS - Motor Response'].values):
    if np.mod(i,6)==0 and i < 65:
        plt.annotate(txt, (ce.HOURS[ce.LABEL=='GCS - Motor Response'].values[i],150),fontsize=14)

for i, txt in enumerate(ce.VALUE[ce.LABEL=='GCS - Verbal Response'].values):
    if np.mod(i,6)==0 and i < 65:
        plt.annotate(txt, (ce.HOURS[ce.LABEL=='GCS - Verbal Response'].values[i],145),fontsize=14)

plt.title('Vital signs and Glasgow Coma Scale over time from admission',fontsize=16)

plt.xlabel('Time (hours)',fontsize=16)
plt.ylabel('Heart rate or GCS',fontsize=16)
plt.ylim(10,165)


#### Task 3

- How is the patient's consciousness changing over time?

## What about the patient's other vital signs?

We can visualise on a single plot several vital signs for out patient.

#### Plot 4: Were the patient's other vital signs stable?

In [ ]:
plt.figure(figsize=(14, 10))

plt.plot(ce.index[ce.LABEL=='Heart Rate'], 
         ce.VALUENUM[ce.LABEL=='Heart Rate'],
         'rx', markersize=8, label='HR')

plt.plot(ce.index[ce.LABEL=='O2 saturation pulseoxymetry'], 
         ce.VALUENUM[ce.LABEL=='O2 saturation pulseoxymetry'], 
         'g.', markersize=8, label='O2')

plt.plot(ce.index[ce.LABEL=='Non Invasive Blood Pressure mean'], 
         ce.VALUENUM[ce.LABEL=='Non Invasive Blood Pressure mean'], 
         'bv', markersize=8, label='MAP')

plt.plot(ce.index[ce.LABEL=='Respiratory Rate'], 
         ce.VALUENUM[ce.LABEL=='Respiratory Rate'], 
         'k+', markersize=8, label='RR')

plt.title('Vital signs over time from admission')
plt.ylim(0,130)
plt.legend()

## Laboratory measurements

Using Pandas 'read_csv function' again, we'll now load the labevents data.
This data corresponds to measurements made in a laboratory - usually on a sample of patient blood. 

In [ ]:
# OPTION 1: load labevents data using the database connection
query = """
SELECT de.subject_id
  , de.charttime
  , di.label, de.value, de.valuenum
  , de.uom
FROM labevents de
INNER JOIN d_labitems di
  ON de.itemid = di.itemid
where de.subject_id = 40080
"""

le = pd.read_sql_query(query,conn)

# OPTION 2: load labevents from the CSV file
# le = pd.read_csv('data/example_labevents.csv', index_col='HOURSSINCEADMISSION')

In [ ]:
# preview the labevents data
le.head()

In [ ]:
# preview the ioevents data
le[le.LABEL=='HEMOGLOBIN']

#### PLot 5: hemoglobin and hematocrit values

In [ ]:
plt.figure(figsize=(14, 10))

plt.plot(le.index[le.LABEL=='HEMATOCRIT'], 
         le.VALUENUM[le.LABEL=='HEMATOCRIT'], 
         'go', markersize=6, label='Haematocrit')

plt.plot(le.index[le.LABEL=='HEMOGLOBIN'], 
         le.VALUENUM[le.LABEL=='HEMOGLOBIN'], 
         'bv', markersize=8, label='Hemoglobin')

plt.title('Laboratory measurements over time from admission')
plt.ylim(0,38)
plt.legend()

# PART 2: Building a predictive model

- Since the MIMIC demo dataset only contains data from 8 patients, all non-survivors, this dataset is not suitable to build mortality prediction models.
- Instead, let's try something with the available data: can we predict SpO2 given arterial pO2 and mean blood pressure?
- First, let’s extract a suitable dataset. We’ll run 3 sub-queries (one for each feature) then combine them.


In [ ]:
query = """

with t1 as -- all time steps
(

SELECT distinct de.subject_id
  , round((strftime('%s',de.charttime)-strftime('%s',ie.intime))/60.0/120.0) as HOURS
FROM chartevents de
INNER join icustays ie
ON de.subject_id = ie.subject_id
order by de.subject_id, charttime

), t2 as  -- po2
(
SELECT de.subject_id
  --, de.charttime
  , round((strftime('%s',de.charttime)-strftime('%s',ie.intime))/60.0/120.0+1) as HOURS
 -- , di.label, de.value
  ,de.valuenum as po2
  --, de.uom
FROM labevents de
INNER JOIN d_labitems di
  ON de.itemid = di.itemid
INNER join icustays ie
ON de.subject_id = ie.subject_id
where label = 'PO2'
order by de.subject_id, hours

), t3 as  -- spo2
(
with t1 as
(SELECT de.subject_id
  , round((strftime('%s',de.charttime)-strftime('%s',ie.intime))/60.0/120.0+1) as HOURS
  , de.valuenum
FROM chartevents de
INNER join d_items di
ON de.itemid = di.itemid
INNER join icustays ie
ON de.subject_id = ie.subject_id
WHERE label = 'O2 saturation pulseoxymetry'
ORDER BY charttime)
select subject_id, hours, avg(valuenum) as spo2
from t1
group by subject_id, hours
order by subject_id, hours
), t4 as -- map
(
with t1 as
(SELECT de.subject_id
  , round((strftime('%s',de.charttime)-strftime('%s',ie.intime))/60.0/120.0+1) as HOURS
  , de.valuenum
FROM chartevents de
INNER join d_items di
ON de.itemid = di.itemid
INNER join icustays ie
ON de.icustay_id = ie.icustay_id
WHERE label = 'Non Invasive Blood Pressure mean'
ORDER BY charttime)
select subject_id, hours, avg(valuenum) as map
from t1
group by subject_id, hours
order by subject_id, hours

)

select t1.*, t2.po2 as po2,t4.map as map, t3.spo2 as spo2
from t1, t2, t3, t4
where t1.subject_id=t2.subject_id and t1.subject_id=t3.subject_id   and t1.subject_id=t4.subject_id   and t4.hours>=t1.hours-2 and t4.hours<=t1.hours+2  and t2.hours>=t1.hours-2  and t2.hours<=t1.hours+2 and t1.hours=t3.hours"""
    
# Run the query  
uo = pd.read_sql_query(query,conn)

# Save the correct columns into X (input) and Y (output) variables
X=uo.drop('subject_id', axis = 1)
X=X.drop('HOURS',axis=1)
Y=uo.spo2
# Save the name of the column headers 
feature_list = list(X.columns)
# Visualise the dataset created 
uo.head()
   

It is good practice to train and test any statistical model on separate data, otherwise the model performance may be overestimated. So, let’s split the dataset into training and testing set.


In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
Xtrain, Xtest, Ytrain , Ytest = train_test_split(X,Y, test_size = 0.2, random_state = 1)

print('Training Features Shape:', Xtrain.shape)
print('Training Labels Shape:', Ytrain.shape)
print('Testing Features Shape:', Xtest.shape)
print('Testing Labels Shape:', Ytest.shape)


Now that we’ve prepared the data, let’s build the predictive model. We’ll be using a random forest model, and learn the relationship between input data (pO2 and MAP) and the output (SpO2). Then, we’ll try the model on the test data.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import roc_auc_score
#from sklearn.neighbors  import KNeighborsClassifier
#from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import matplotlib

# Build a random forest classifier
rf = RandomForestClassifier()
rf.fit(Xtrain,Ytrain.astype('int'))

# Use the random forest predict method on the test data
predictions = rf.predict(Xtest)


Let’s assess the performance of the model we built. We can measure the absolute error between actual and predicted values, and the mean absolute percentage error (MAPE).

In [ ]:
# Calculate the absolute errors
errors = abs(predictions - Ytest)
print('Mean Absolute Error:', round(np.mean(errors), 2), '% of SpO2')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / Ytest)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')



#### Plot 6: Actual versus predicted values.

In [ ]:
# Plot actual versus predicted values
plt.scatter(Xtest.po2, Ytest, c="g", alpha=0.5, label="True test data")
plt.scatter(Xtest.po2, predictions, c="r", alpha=0.5, label="Predicted data")

plt.xlabel("pO2")
plt.ylabel("spO2")
plt.legend(loc=4)
plt.show()


# Conclusion

This concludes our workshop. To summarise, we gained insight into MIMIC-III, the publicly accessible critical care database. We explored raw patient data with SQL queries, extracted and analysed the data of a single patient and built a simple predictive model.

If after this workshop you would like to gain access to the full MIMIC-III dataset, which contains rich data for over 40,000 patients, please see: https://mimic.physionet.org/gettingstarted/access/